# Basics of text processing

### Natural Language Processing and Information Extraction,  2021 WS
10/15/2021

Gábor Recski

## In this lecture
- Regular Expressions

- Text segmentation and normalization:
   - sentence splitting and tokenization
   - lemmatization, stemming, decompounding, morphology

## Import dependencies

In [ ]:
import re
from collections import Counter

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
import stanza
stanza.download('en')
stanza.download('de')

## Regular expressions

### Basics

![re1](media/re1.png)([SLP Ch.2](https://web.stanford.edu/~jurafsky/slp3/2.pdf))

In [ ]:
text = open('data/alice.txt').read()
print(text[:100])

In [ ]:
re.search('Alice', text)

In [ ]:
text[35:40]

![re2](media/re2.png)([SLP Ch.2](https://web.stanford.edu/~jurafsky/slp3/2.pdf))

In [ ]:
re.search('[Rr]abbit', text)

In [ ]:
re.findall('[Rr]abbit', text[:5000])

In [ ]:
for match in re.finditer('[Rr]abbit', text[:5000]):
    print(match.group(), match.span())

![re3](media/re3.png)([SLP Ch.2](https://web.stanford.edu/~jurafsky/slp3/2.pdf))

In [ ]:
re.findall(' [A-Za-z][a-z][a-z] ', text[:5000])

In [ ]:
Counter(re.findall(' [A-Za-z][a-z][a-z] ', text)).most_common(10)

![re4](media/re4.png)([SLP Ch.2](https://web.stanford.edu/~jurafsky/slp3/2.pdf))

![re5](media/re5.png)([SLP Ch.2](https://web.stanford.edu/~jurafsky/slp3/2.pdf))

![re6](media/re6.png)([SLP Ch.2](https://web.stanford.edu/~jurafsky/slp3/2.pdf))

In [ ]:
re.findall('...', text[:100])

![re7](media/re7.png)([SLP Ch.2](https://web.stanford.edu/~jurafsky/slp3/2.pdf))

In [ ]:
re.findall('\w', text[:50])

In [ ]:
re.split('\s', text[:100])

![re8](media/re8.png)([SLP Ch.2](https://web.stanford.edu/~jurafsky/slp3/2.pdf))

In [ ]:
re.findall('\w+', text[:100])

In [ ]:
Counter(re.findall('\w+', text)).most_common(20)

In [ ]:
Counter(re.findall('[^\w\s]', text)).most_common(20)

### Substitution and groups

In [ ]:
re.sub('\s+', ' ', text[:100])

In [ ]:
print(re.sub('\s+', '\n', text[:100]))

In [ ]:
re.findall('CHAPTER [^\s]+', text)

In [ ]:
print(re.sub('CHAPTER ([^\s]+)', 'Chapter \\1', text[:100]))

In [ ]:
print(re.sub('CHAPTER ([^\s.]+).\n([^\n]*)', 'Chapter \\1: \\2', text[:100]))

In [ ]:
re.findall('CHAPTER ([^\s.]+).\n([^\n]*)', text)

Regular expressions are surprisingly powerful. Also, with the right implementation, they are literally as fast as you can get. That's because they are equivalent to [finite state automata (FSAs)](https://en.wikipedia.org/wiki/Finite-state_machine). Actually, every regular expression is a [regular grammar](https://en.wikipedia.org/wiki/Regular_grammar) defining a [regular language](https://en.wikipedia.org/wiki/Regular_language).

![re_xkcd](media/re_xkcd.png)([XKCD #208](https://xkcd.com/208/))

## Text segmentation

### Sentence splitting

#### How to split a text into sentences?

In [ ]:
text2 = "'Of course it's only because Tom isn't home,' said Mrs. Parsons vaguely."

Naive: split on `.`, `!`, `?`, etc.

In [ ]:
re.split('[.!?]', text2)

Better: use language-specific list of abbreviation words, collocations, etc.

In [ ]:
nltk.sent_tokenize(text2)

Custom lists of patterns are often necessary for special domains. 

In [ ]:
text3 = "An die Stelle der Landesgesetze vom 17. Jänner 1883, n.ö.L.G. u. V.Bl. Nr. 35, vom 26. Dezember 1890, n.ö.L.G. u. V.Bl. Nr. 48, vom 17. Juni 1920 n.ö.L.G. u. V.Bl. Nr. 547, vom 4. November 1920 n.ö.L.G. u. V.Bl. Nr. 808, und vom 9. Dezember 1927, L.G.Bl. für Wien Nr. 1 ex 1928, die, soweit dieses Gesetz nichts anderes bestimmt, zugleich ihre Wirksamkeit verlieren, hat die nachfolgende Bauordnung zu treten."

In [ ]:
print(text3)

In [ ]:
nltk.sent_tokenize(text3, language='german')

###  Tokenization

#### How to  split text into words?

#### Naive approach: split on whitespace

In [ ]:
text2.split()

#### Better: separate punctuation marks

In [ ]:
re.findall('(\w+|[^\w\s]+)', text2)[:30]

#### Best: add some language-specific conventions:

In [ ]:
nltk.word_tokenize(text2)

## Text normalization

In [ ]:
words = nltk.word_tokenize(text)

In [ ]:
words[:10]

In [ ]:
Counter(words).most_common(10)

Let's get rid of punctuation

In [ ]:
words = [word for word in words if re.match('\w', word)]

In [ ]:
Counter(words).most_common(10)

Filtering common function words is called __stopword removal__

In [ ]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
print(stopwords)

In [ ]:
words = [word for word in words if word.lower() not in stopwords]

In [ ]:
Counter(words).most_common(20)

### Lemmatization and stemming

Words like _say_, _says_, and _said_ are all different **word forms** of the same **lemma**. Grouping them together can be useful in many applications. 

**Stemming** is the reduction of words to a common prefix, using simple rules that only work some of the time:

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
for word in ('say', 'says', 'said'):
    print(stemmer.stem(word))

In [ ]:
for word in ('he', 'his', 'him'):
    print(stemmer.stem(word))

**Lemmatization** is the mapping of word forms to their lemma, using either a dictionary of word forms, a grammar of how words are formed (a **morphology**), or both.

In [ ]:
nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos')

In [ ]:
doc = nlp(text)

In [ ]:
for sentence in doc.sentences[:5]:
    for word in sentence.words:
        print(word.text + '\t' + word.lemma)
    print()

Now we can count lemmas

In [ ]:
Counter(
    word.lemma for sentence in doc.sentences for word in sentence.words
    if word.lemma not in stopwords and re.match('\w', word.lemma)).most_common(20)

The full analysis of how a word form is built from its lemma is known as **morphological analysis**

In [ ]:
for sentence in doc.sentences[:5]:
    for word in sentence.words:
        print('\t'.join([word.text, word.lemma, word.upos, word.feats if word.feats else '']))
    print()

A special case of lemmatization is **decompounding**, recognizing multiple lemmas in a word

In [ ]:
nlp('roller-coaster')

In [ ]:
nlp('wastebasket')

For English you might say that this is good enough... but _some languages_ allow forming compounds on the fly...

In [ ]:
nlp_de = stanza.Pipeline('de', processors='tokenize,lemma,pos')

In [ ]:
nlp_de('Kassenidentifikationsnummer')

There is no good generic solution and no standard tool. There are some unsupervised approaches like [SECOS](https://github.com/riedlma/SECOS) and [CharSplit](https://github.com/dtuggener/CharSplit), and there are also full-fledged morphological analyzers that might work, like [SMOR](https://www.cis.lmu.de/~schmid/tools/SMOR/) and its extensions [zmorge](https://pub.cl.uzh.ch/users/sennrich/zmorge/) and [SMORLemma](https://github.com/rsennrich/SMORLemma).

## Examples

### Text processing with regular expressions

Load a sample text

In [ ]:
text = open('data/alice.txt').read()
print(text[:1000])

In [ ]:
def clean_text(text):
    cleaned_text = re.sub('_','',text)
    cleaned_text = re.sub('\n', ' ', cleaned_text)
    return cleaned_text

In [ ]:
text = clean_text(text)

In [ ]:
print(text[:1000])

Let's split this into sentences, then words.

In [ ]:
sens = sent_tokenize(text)

In [ ]:
print('\n\n'.join(sens[:5]))

In [ ]:
toks = [word_tokenize(sen) for sen in sens]

In [ ]:
print('\n\n'.join('\n'.join(sen) for sen in toks[:5]))

Let's also write this to a file

In [ ]:
with open('data/alice_tok.txt', 'w') as f:
    f.write('\n\n'.join('\n'.join(sen) for sen in toks) + '\n')

Let's try to find all names using regexes

In [ ]:
def find_names(toks):
    curr_name = []
    for sen in toks:
        for tok in sen[1:]:
            if re.match('[A-Z][a-z]+', tok):
                curr_name.append(tok)
            elif curr_name:
                yield ' '.join(curr_name)
                curr_name = []
                
        if curr_name:
            yield curr_name
            
        
def count_names(toks):
    name_counter = Counter()
    
    for name in find_names(toks):
        name_counter[name] += 1
    
    for name, count in name_counter.most_common():
        print(name, count)

In [ ]:
count_names(toks)

We can filter our tokens for stopwords:

In [ ]:
toks_without_stopwords = [[tok for tok in sen if tok.lower() not in stopwords] for sen in toks]

In [ ]:
print('\n\n'.join('\n'.join(sen) for sen in toks_without_stopwords[:5]))

In [ ]:
count_names(toks_without_stopwords)

Let's also write the stopwords into a file

In [ ]:
with open('data/stopwords.txt', 'w') as f:
    f.write('\n'.join(sorted(stopwords)) + '\n')

Continue to [Text processing on the Linux command line](https://github.com/tuw-nlp-ie/tuw-nlp-ie-2021WS/blob/main/lectures/01_Text_processing/01b_Text_processing_Linux_command_line.ipynb)